<h1>Code for Experiment 311</h1>

This notebook contains all of the code used to process the data in Experiment 311.

<h2>Importing data and modules</h2>

The modules used in this code were numpy, matplotlib.pyplot and the gamma function from scipy.special.

Firstly, the data needed to be read into Python as a numpy array. This was complicated by the fact that the data was in five minute chunks. To combine the data, it was read into a list as separate arrays, modified so that it contained the correct time, and then concatenated together.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import gamma

B = []
for i in range(14):
    b0 = np.zeros((3000000, 3))
    b0[:, 0] = i*np.ones(3000000)*3000000 # Shifting counts accordingly.
    b0[:, 1] = i*np.ones(3000000)*300 # Shifting time accordingly.
    # B += [np.loadtxt(f".\\Data\\NuclearCounting{i+1}.txt", delimiter=', ') + b0]  # Importing and putting data into a list.
    B += [np.loadtxt(f"C:\\Users\\bayle\\OneDrive\\Documents\\University Documents\\2022 S1\\PHYSICS 390\\Experiment 311\\Data\\NuclearCounting{i+1}.txt", delimiter=', ') + b0]  # Importing and putting data into a list.

A = B[0]
for i in range(13):
    A = np.concatenate((A, B[i+1])) # Iteratively combining the data.
count = A[:, 0]
time = A[:, 1]
volt = A[:, 2]

Next, the peaks in the voltage needed to be recorded. As pointed out in the report, a deviation of $4\sigma$ from the mean seemed to be adequate to capture the peaks.

To encode peaks, the array `countfin` was constructed, which was essentially a function of the count: if a peak had started at the input count, then the output was 1, and otherwise it was 0. The array was constructed by iterating through the counts, first asking if the voltage was below $\mu - 4\sigma$, and second asking if the previous count was below $\mu - 4\sigma$. The second step was done by creating a toggle, which turned on when the threshold was reached.

To encode distances between peaks, `countd` was constructed. This was essentially done by iterating through countfin, finding when it was 1, and adding the corresponding count to `countd`.

In [ ]:
mean = np.mean(volt)
sigma = np.std(volt)

countfin = np.zeros(len(count)) # Initializing countfin.

First = 1 # Toggle.

for i in range(len(count)): 
    if volt[i] <= mean - 4*sigma: # If the signal is significantly low,
        if NotFirst == 1: # and this is the first time in a while,
            countfin[i] = 1 # then a peak has started.
            First = 0 # No longer the first count below the peak.
    if volt[i] > mean - 4*sigma: # When the peak ends,
        First = 1 # the next dip below the threshold will be the first.

# The last array records the exact counts at which peaks are.
countd = np.array([i for i in range(len(countfin)) if countfin[i]==1])

<h2>Constancy Check</h2>

Before the data could be interpreted, the constancy of the rate of counts needed to be verified. In this section the data was divided into 60-second intervals, and the number of counts in each interval was recorded. Then a line was fitted to the data, and the result was plotted.

In [ ]:
Tc = 60 # Time interval for the constancy check

N = int(time[-1]/Tc) # The number of intervals
cnum = int(count[-1]/N) # The number of datapoints per interval

peakcount = np.zeros(N) # peakcount is the no. of peaks in each interval.
for i in range(N):
    ci = i*cnum # Starting count for the ith interval
    peakcount[i] = np.sum(countfin[ci:(ci+cnum)])

Z = np.arange(N)
p = np.polyfit(Z, parray, 1) # Fitted line
print(f'The slope of the line is {p[0]:.3g}.')

plt.figure()
plt.title('Counts per Minute')
plt.xlabel('Time (min)')
plt.ylabel('Counts')
plt.plot(Z, peakcount, label='Data')
plt.plot(Z, Z*p[0] + p[1], color='red', label='Fitted')
plt.legend()
plt.show()

<h2>Poisson Distribution</h2>

The Poisson distribution was the first that was tested, and to do so an interval length needed to be chosen. The distribution has a distinctive shape when the mean counts per interval is between 3 and 6, so the time interval was chosen so that this number was 4.5. The rest of the data analysis is self-explanatory.

In [ ]:
mcr = np.sum(countfin)/time[-1] # Mean count rate; number of counts over time.
Tp = 4.5/mcr # explained above
Np = int(time[-1]/Tp) # Total number of intervals.
cnump = int(count[-1]/N) # Number of datapoints in each interval.

peakcountp = np.zeros(Np) # peakcountp is the no. of peaks within an interval.
for i in range(Np):
    ci = i*cnump
    peakcountp[i] = np.sum(countfin[ci:(ci+cnump)])

countmeanp = np.mean(peakcountp) # Mean number of peaks in each interval.
countstdp = np.std(peakcountp) # SD of the number of peaks in each interval.
print(f'The mean of the distribution is {countmean:.4g}, and the variance is {countstd**2:.4g}.')

def f(x, mu, sigma): # Expected PDF for a Gaussian.
    return 1/(sigma*np.sqrt(2*np.pi))*np.exp(-((x - mu)/sigma)**2)

def PMF(k, lam): # Poisson PMF, made continuous using the gamma function.
    return lam**k*np.exp(-lam)/gamma(k+1)

# Plotting.
plt.figure()
plt.title('Histogram of Counts in 0.68s Intervals')
plt.xlabel('Number of Counts in Interval')
plt.ylabel('Proportion of Intervals')
plt.hist(peakcountp, bins=int(max(peakcountp) - min(peakcountp)), density=True, rwidth=0.9, align='left', label='Measured')
plt.plot(np.arange(min(peakcountp), max(peakcountp), 0.01), \
         PMF(np.arange(min(peakcountp), max(peakcountp), 0.01), countmeanp),\
         color='red', label='Predicted')
plt.legend()
plt.show()

plt.figure()
plt.title('Gaussian Fit to 0.68s Intervals')
plt.xlabel('Number of Counts in Interval')
plt.ylabel('Proportion of Intervals')
plt.hist(peakcountp, bins=int(max(peakcountp) - min(peakcountp)), density=True, rwidth=0.9, align='left', label='Measured')
plt.plot(np.arange(min(peakcountp), max(peakcountp), 0.01), \
         f(np.arange(min(peakcountp), max(peakcountp), 0.01), countmeanp, countstdp), \
         color='red', label='Gaussian')
plt.legend()
plt.show()

# Implementation of the trapezoid method to find the expected number of intervals
# with a certain number of peaks i, according to the Poisson distribution (wrong=0)
# and the Gaussian distribution (wrong=1).
def expect(i, wrong=0): 
    if wrong==0:
        return PMF(i, countmeanp)
    else:
        return 0.5*(f(i, countmeanp, countstdp) + f(i+1, countmeanp, countstdp))

# Number of considered peak counts (since the others are all zero).
binsp = max(peakcountp) - min(peakcountp)

# histarray is an array of bins.
histarray = np.arange(min(peakcountp), binno+min(peakcountp))
Ep = expect(histarray)
Ewrong = expect(histarray, wrong=1)

def observed(array): # Number of observed intervals with a given peak count, as in 'array'.
    k = np.zeros(len(array))
    for i in range(len(array)): # Running through peak counts.
        for j in range(len(peakcountp)): # Running through intervals.
            if peakcountp[j] == array[i]: # If the number of peaks in the examined interval is equal to the desired peak count array[i],
                k[i] += 1 # increment the number in the ith position by 1.
    return k # The output of the function is as follows: for the ith peak count in array, k[i] is the number of intervals containing array[i] peaks.

O = observed(histarray)/Np # Normalizing.

chisquarep = np.sum((O - Ep)**2/Ep) # Chisquare value according to Poisson
chisquarewrong = np.sum((O - Ewrong)**2/Ewrong) # Chisquare value according to Gaussian
print(f'The \u03C7\u00B2 value for the Poisson fit is {chisquarep:.4g}.')
print(f'The \u03C7\u00B2 value for the Gaussian fit is {chisquarewrong:.4g}.')

<h2>Gaussian Distribution</h2>

The Gaussian distribution was the second to be tested. Since the data only appears Gaussian for large mean counts, the interval was chosen so that the mean count per interval was between 200 and 300.

In [ ]:
Tg = 240/mcr # explained above
Ng = int(time[-1]/Tg) # Total number of intervals.
cnumg = int(count[-1]/N) # Number of datapoints in each interval.

peakcountg = np.zeros(Ng) # peakcountp is the no. of peaks within an interval.
for i in range(Ng):
    ci = i*cnumg
    peakcountg[i] = np.sum(countfin[ci:(ci+cnumg)])

countmeang = np.mean(peakcountg) # Mean number of peaks in each interval.
countstdg = np.std(peakcountg) # SD of the number of peaks in each interval.
print(f'The mean of the distribution is {countmean:.4g}, and the variance is {countstd**2:.4g}.')

# Plotting.
plt.figure()
plt.title('Histogram of Counts in 36s Intervals')
plt.xlabel('Number of Counts in Interval')
plt.ylabel('Proportion of Intervals')
plt.hist(peakcountg, bins=int(max(peakcountg) - min(peakcountg)), density=True, rwidth=0.9, align='left', label='Measured')
plt.plot(np.arange(min(peakcountg), max(peakcountg), 0.01), \
         f(np.arange(min(peakcountp), max(peakcountg), 0.01), countmeang, countstdg), color='red', label='Predicted')
plt.legend()
plt.show()

def expectg(i): 
    return 0.5*(f(i, countmeang, countstdg) + f(i+1, countmeang, countstdg))

# Number of considered peak counts (since the others are all zero).
binsg = max(peakcountg) - min(peakcountg)

# histarray is an array of bins.
histarray = np.arange(min(peakcountg), binno+min(peakcountg))
Eg = expectg(histarray)

def observed(array): # Number of observed intervals with a given peak count, as in 'array'.
    k = np.zeros(len(array))
    for i in range(len(array)): # Running through peak counts.
        for j in range(len(peakcountg)): # Running through intervals.
            if peakcountg[j] == array[i]: # If the number of peaks in the examined interval is equal to the desired peak count array[i],
                k[i] += 1 # increment the number in the ith position by 1.
    return k # The output of the function is as follows: for the ith peak count in array, k[i] is the number of intervals containing array[i] peaks.

O = observed(histarray)/Ng # Normalizing.

chisquareg = np.sum((O - Eg)**2/Eg) # Chisquare value according to Gaussian
print(f'The \u03C7\u00B2 value for the Gaussian fit is {chisquareg:.4g}.')

<h2>Exponential Distribution</h2>

The exponential distribution was the third to be tested. The bin size was chosen to be 0.25s, simply because it best represented the data.

In [ ]:
countdiff = np.zeros(len(countd) - 1) # countdiff is an array of count differences between peaks.
for i in range(len(countdiff)):
    countdiff[i] = countd[i+1] - countd[i] # Taking differences of all peak counts.
timediff = countdiff/10000 # Sample rate of 10kHz; timediff is time differences between peaks.

diffmean = np.mean(timediff) # Mean time difference.
diffstd = np.std(timediff) # Standard deviation of time difference.

tau = 1/mcr # by definition of mean time interval
def W(t): # Expected PDF for an exponential distribution.
    return (1/tau)*np.exp(-t/tau)

# Plotting.
t = np.arange(0, max(timediff), 0.01)

plt.figure()
plt.title('Histogram of Intervals between Decays')
plt.xlabel('Time (s)')
plt.ylabel('Proportion of Intervals')
plt.hist(timediff, bins=np.arange(0, 2, 0.1), density=True, rwidth=0.9, label='Measured')
plt.plot(t, W(t), color='red', label='Predicted')
plt.legend()
plt.show()

def expect(i): # Use of the trapezoid rule to find the expected probability.
    return 0.5*(W(i) + W(i+0.1))

binsexp = int(2/0.1) # Number of bins.

histarray = np.arange(0, 2, 0.1)
E = expect(histarray)

def observed(array):
    k = np.zeros(len(array))
    for i in range(len(array) - 1):
        for j in range(len(timediff)):
            if timediff[j] >= array[i] and timediff[j] < array[i+1]:
                k[i] += 1
    
    return k

O = observed(X)/len(timediff)*10

chisquareexp = np.sum((O-E)**2/E)
print(f'The \u03C7\u00B2 value for the Gaussian fit is {chisquareexp:.4g}.')

<h2>Critical $\chi^2$ Values</h2>

To calculate the critical $\chi^2$ values for the report, the quantile function for the $\chi^2$ distribution was constructed directly. This was done using the PDF for the distribution given in [1].

In [ ]:
from scipy.special import gamma
from scipy.optimize import root_scalar
from scipy.integrate import quad

def chisq(x, k): # There are two definitions: one for numbers, and one for arrays. This was primarily done for testing.
    if type(x) == float or type(x) == int:
        if x == 0:
            return 0
        return x**(k/2-1)*np.exp(-x/2)/(gamma(k/2)*2**(k/2))
    if type(x) == np.ndarray:
        ar = np.zeros(len(x))
        for i in range(len(x)):
            if x[i] == 0:
                ar[i] = 0
            else:
                ar[i] = x[i]**(k/2-1)*np.exp(-x[i]/2)/(gamma(k/2)*2**(k/2))
        return ar
    

def CDF(x, k):
    if type(x) == float or type(x) == int:
        return quad(chisq, 0, x, args=(k))[0]
    if type(x) == np.ndarray:
        ar = np.zeros(len(x))
        for i in range(len(x)):
            ar[i] = quad(chisq, 0, x[i], args=(k))[0]
        return ar

def f(x, k, alpha): # The function which is zero at the relevant quantile.
    return CDF(x, k) - 1 + alpha

def quantile(alpha, k, x0=0, x1=100):
    return root_scalar(f, args=(k, alpha), x0=x0, x1=x1) # The secant method was used to construct the quantile function.

<h2>Addenda</h2>

There were some extra graphs drawn, for illustration purposes or for extra analysis. The first cell shows the graphs for the first decay and the first three seconds of data, while the second cell analyses the background radiation collected over 5 minutes.

In [ ]:
m = 2400
M = 2650

plt.figure()
plt.title('Single Decay')
plt.plot(time[m:M], volt[m:M])
plt.xlabel('Time (s)')
plt.ylabel('Voltage (V)')
plt.grid()
plt.show()

plt.figure()
plt.title('First Three Seconds of Data')
plt.plot(time[:30000], volt[:30000])
plt.xlabel('Time (s)')
plt.ylabel('Voltage (V)')
plt.grid()
plt.show()

In [ ]:
C = np.loadtxt(".\\Data\\NuclearCountingAmb.txt", delimiter=', ')
count0 = C[:,0]
time0 = C[:,1]
volt0 = C[:,2]

mean0 = np.mean(volt0)
sigma0 = np.std(volt0)

countfin0 = np.zeros(len(count0)) # Countfin is a peak array, as above.

First = 1

for i in range(len(count0)):
    if volt0[i] <= mean0 - 4*sigma0:
        if First == 1:
            countfin0[i] = 1
            First = 0
    if volt0[i] > mean0 - 4*sigma0:
        First = 1

plt.figure()
plt.plot(count0[:100000], volt0[:100000])
plt.show()

print(np.sum(countfin0))